In [ ]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.nn import functional as F

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
!nvidia-smi

In [ ]:
import wandb
%env "WANDB_NOTEBOOK_NAME" "demo_wine_wandb_test"
wandb.login()

In [ ]:
df = pd.read_csv("./data/wine_data.csv")

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.describe()

# DATA WRANGLING

check for nulls and duplicates

In [ ]:
df.isna().sum()

In [ ]:
df.duplicated().sum()

# MACHINE LEARNING
### ML PREP

In [ ]:
# Encode target labels with value between 0 and n_classes-1.

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report


In [ ]:
le = LabelEncoder()
df['Class'] = le.fit_transform(df['Class'])
df.sample(10)

In [ ]:
df['Class'].unique()

### SEPARATE FETURES AND TARGET

In [ ]:
df_features = df.drop('Class', axis=1)
df_features.head()

In [ ]:
df_target = df[['Class']]
df_target.head()

In [ ]:
#SPLIT DATA
from sklearn.model_selection import train_test_split

In [ ]:
X_train, x_test, Y_train, y_test = train_test_split(df_features, 
                                                    df_target,
                                                    test_size=0.3,
                                                     random_state=42)

In [ ]:
X_train.shape, x_test.shape,

In [ ]:
 Y_train.shape, y_test.shape

### Convert data to Tensors for Pytorch

In [ ]:
Xtrain = torch.from_numpy(X_train.values).float()
Xtest = torch.from_numpy(x_test.values).float()
print(Xtrain.shape, Xtest.shape)

In [ ]:
print(Xtrain.dtype, Xtest.dtype)

We have successfully converted our  X_data into torch tensors of float32 datatype

In [ ]:
Ytrain = torch.from_numpy(Y_train.values).view(1,-1)[0]
Ytest = torch.from_numpy(y_test.values).view(1, -1)[0]
print(Ytrain.shape, Ytest.shape)

We use the **view()** to reshape the tensor.<br>
The loss function doesn't support multi-target and therefore, we should use a 1D Tensor of 1 row containing the labels.<br>
We have successfully converted our y_data

In [ ]:
print(Ytrain.dtype, Ytest.dtype)

## PyTorch
### We create a classifier and define our neural network for our model

### Hyperparameterss

In [ ]:
input_size = 13
output_size = 3
hidden_size = 100

In [ ]:
config = dict(
                input_size = 13,
                output_size = 3,
                hidden_size = 100,
                dataset = "wine dataset",
                architecture = 'Linear', 
                learning_rate = 0.01,
                loss = nn.NLLLoss(),
)


In [ ]:
for k,v in config.items():
    print(k, v)

### Define the neural network


In [ ]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)

    def forward(self, X):
        X = torch.sigmoid((self.fc1(X)))
        X = torch.sigmoid(self.fc2(X))
        X = self.fc3(X)

        return F.log_softmax(X, dim=-1)

In [ ]:
model = Net()
# preview out model
model

### Define Optimizer and Loss Function

In [ ]:
import torch.optim as optim

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=config.get("learning_rate"))
loss_fn = config.get("loss")

In [ ]:
epochs = 1000
with wandb.init(project="demo_wandb_test", config = config):
    wandb.watch(model, criterion=None, log="gradients", log_freq=10)
    #here
    for epoch in range(epochs):
        #HERE
        optimizer.zero_grad()
        Ypred = model(Xtrain)

        loss = loss_fn(Ypred, Ytrain)
        loss.backward()

        optimizer.step()

        wandb.log({'Epoch': epoch, "Loss": loss.item(), "Predicted Values" : Ypred.values})


    # SAVE MODEL STATE DICT TO DISK

    wandb.save(torch.save(model.state_dict(), "./models/home_state_dict.pt"))

    #LOAD MODEL FROM DISK and EVALUATE
    new_model =  Net()
    new_model.load_state_dict(torch.load("./models/home_state_dict.pt"))
    new_model.eval()

    # SET THE PREDICTIONS
    predict = new_model(Xtest)
    _, predict_y = torch.max(predict, 1)

    # Visualize Confusion Matrix

    wandb.sklearn.plot_confusion_matrix(Ytest, predict_y, labels = [0,1,2])
    # Print Metrics
    wandb.log({"accuracy_score" : accuracy_score(Ytest, predict_y),
    "precision_score" : precision_score(Ytest, predict_y, average='weighted'),
    "recall_score": recall_score(Ytest, predict_y, average="weighted")})
    wandb.run.summary()
wandb.finish()
